<a href="https://colab.research.google.com/github/WeizmannML/course2020/blob/master/Tutorial2/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <font color=blue>Pytorch allows us to use automatic differentiation & use the power of GPU </font>
#### All the code snippets in the following example are taken from https://pytorch.org/tutorials/beginner/pytorch_with_examples.html 
#### Other libraries like tensorflow or JAX allows us to do the same. 

In [0]:
from __future__ import print_function
import torch

In [0]:
torch.__version__

'1.5.0'

In [0]:
import os, sys
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
# ----- for automatic detection of CPU vs GPU device ---------- #
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )

#### The basic working objects are torch.Tensors 

In [0]:
a = torch.tensor([[1., -1.], [1., -1.]])

In [0]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))

In [0]:
type(a), type(b)

(torch.Tensor, torch.Tensor)

In [0]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [0]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)      # new_* methods take in sizes
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [0]:
x.device ### equivalently can be done as x = x.to(cuda_device)

device(type='cpu')

In [0]:
x = torch.randn_like(x, dtype=torch.float64)    # override dtype!
print(x) 

tensor([[ 1.7977, -1.2452,  0.5304],
        [-1.5344, -0.3468, -1.1379],
        [ 0.6575, -0.8318,  0.5323],
        [ 0.6046,  0.8806,  1.3042],
        [-0.0484, -0.5992,  0.3282]], dtype=torch.float64)


In [0]:
print(x.shape), print(x.size())

torch.Size([5, 3])
torch.Size([5, 3])


(None, None)

#### Conversion to numpy arrays

In [0]:
y = x.numpy()

In [0]:
y

array([[ 1.79767662, -1.24517159,  0.53044555],
       [-1.5344497 , -0.34684012, -1.13793026],
       [ 0.65748941, -0.8317851 ,  0.53231995],
       [ 0.60460834,  0.8806115 ,  1.30415491],
       [-0.04838126, -0.59917041,  0.32818938]])

In [0]:
y.dtype

dtype('float64')

## The automatic differentiation of tensors through autograd

In [0]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)

x = torch.randn_like(x, dtype=torch.double, device=cuda_device, requires_grad=True)
print(x)

tensor([[ 0.5102, -0.4021, -0.6409],
        [ 1.9724, -0.2610, -0.3242],
        [-0.0356, -0.1507, -2.6582],
        [ 0.7653, -0.2435,  2.2780],
        [-0.1190,  0.2608,  0.4066]], dtype=torch.float64, requires_grad=True)


In [0]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]), dtype=torch.double, device=cuda_device, requires_grad=True)
print(b)

tensor([[1., 2., 3.],
        [4., 5., 6.]], dtype=torch.float64, requires_grad=True)


In [0]:
y = x + 2
print(y)

tensor([[ 2.5102,  1.5979,  1.3591],
        [ 3.9724,  1.7390,  1.6758],
        [ 1.9644,  1.8493, -0.6582],
        [ 2.7653,  1.7565,  4.2780],
        [ 1.8810,  2.2608,  2.4066]], dtype=torch.float64,
       grad_fn=<AddBackward0>)


In [0]:
y.grad_fn

In [0]:
## -- assigning a required_grad criteria -- #
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [0]:
print(a), print(b)

tensor([[1.5064, 1.1261],
        [0.2980, 1.0568]], requires_grad=True)
tensor(4.7430, grad_fn=<SumBackward0>)


(None, None)

In [0]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([  347.0835, -1292.2808,   221.7831], grad_fn=<MulBackward0>)


In [0]:
x

tensor([ 0.3389, -1.2620,  0.2166], requires_grad=True)

In [0]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([2.0000e-01, 2.0000e+00, 2.0000e-04])


### Doing a backpropagation in the numpy way with torch tensors

In [0]:

dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0) # -- the clamp filters out < 0 values
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 50 == 49:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h) # -- we had x.T for numpy arrays

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

49 19687.40234375
99 1247.0986328125
149 131.6090850830078
199 16.24941635131836
249 2.1392900943756104
299 0.2909495532512665
349 0.04048660770058632
399 0.005967785604298115
449 0.0011125365272164345
499 0.0003139939217362553


## Using the autograd function of pytorch to compute the derivatives & do backpropagation

In [0]:
# -*- coding: utf-8 -*-
dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2) # --- the forward pass through two layers is happening here.

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

49 15529.0771484375
99 752.7603149414062
149 68.41072845458984
199 7.534181118011475
249 0.908591628074646
299 0.11579940468072891
349 0.01547524519264698
399 0.0023776493035256863
449 0.0005157676641829312
499 0.00016332478844560683


### The PyTorch nn Modeule, contains the basic NN layers

In [0]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

49 36.37443923950195
99 2.354414224624634
149 0.26214244961738586
199 0.03899145871400833
249 0.007049467880278826
299 0.0015036026015877724
349 0.0003607187536545098
399 9.672551823314279e-05
449 2.761617724900134e-05
499 8.210541636799462e-06


### Updating the weights through PyTorch optim 

In [0]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

49 191.87445068359375
99 46.8408088684082
149 7.998919486999512
199 1.1330779790878296
249 0.18173131346702576
299 0.03769247978925705
349 0.008523359894752502
399 0.0017651371890679002
449 0.0003159788029734045
499 4.8122674343176186e-05
